# Black Litterman Model

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv("../data/bl_excess_returns_data.csv")

In [21]:
market_weights = np.matrix([[0.5 ], [ 0.4], [ 0.1 ]])

cols = ['US Equity', 'Foreign EQ', 'Emerging EQ']
covariance = data[cols].cov().to_numpy()
market_weights

matrix([[0.5],
        [0.4],
        [0.1]])

In [11]:
risk_aversion = 2.0
tau = 0.05
market_variance = float(np.matmul(market_weights.T, np.matmul(covariance, market_weights)))
std_dev = np.sqrt(market_variance)

market_expected_excess_returns = risk_aversion * market_variance

sharp_ratio = market_expected_excess_returns/std_dev

prior_returns = risk_aversion * np.matmul ( covariance , market_weights)

#covariance * tau
covtau = tau * covariance

In [12]:
view = np.array([[1, 0, 0], [0, 1, -1]])
relative_confidence = 1  # = conf. in view / conf. in prior
tau_omega = tau/relative_confidence
omega = tau_omega*np.matmul(view,np.matmul(covariance , view.T) )
Q = [[0.015], [0.030]]

diagOmega = np.diag(np.diag (omega))
prior_precision = np.matmul(view, np.matmul(tau * covariance , view.T))

In [13]:
posterior_returns = prior_returns + np.matmul(np.matmul(covtau, np.matmul(view.T, np.linalg.inv(prior_precision + omega))), Q-np.matmul(view, prior_returns))

posterior_return_dist = covariance + (covtau - np.matmul(np.matmul(np.matmul(covtau,view.T),np.linalg.inv(prior_precision+omega)),np.matmul(view, covtau)))

## Portfolio Optimization

In [17]:
unconstrained_opt = np.matmul( np.matrix(posterior_returns).T, np.linalg.inv(risk_aversion * posterior_return_dist))
# summing the values of unconstrained to use for constrained
opt_sum = unconstrained_opt.sum(dtype = 'float')
opt_weight = np.matrix( unconstrained_opt / opt_sum).T
expected_returns_opt = float( np.matmul( np.matrix(posterior_returns).T, opt_weight) )
variance_opt =float(np.matmul( np.matmul (opt_weight.T , posterior_return_dist), opt_weight))
std_dev_opt = np.sqrt(variance_opt)
sharpe_ratio_opt = expected_returns_opt/std_dev_opt